In [2]:
import functools
from dataclasses import dataclass, field

@dataclass()
class Machine:
    button_a: tuple[int, int] = ()
    button_b: tuple[int, int] = ()
    prize: tuple[int, int] = ()

def load_input() -> list[Machine]:
    with open("../../data/day13-input.txt") as f:
        machines: list[Machine] = []
        for group in f.read().split('\n\n'):
            machine = Machine()
            rules = group.split('\n')
            parts = rules[0].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_a = (int(parts[0]), int(parts[1]))
            parts = rules[1].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_b = (int(parts[0]), int(parts[1]))
            parts = rules[2].split(':')[1].replace('X=', '').replace('Y=', '').replace(' ', '').split(',')
            machine.prize = (int(parts[0]), int(parts[1]))
            machines.append(machine)
    return machines


In [3]:
machines = load_input()